# Installations & Util

In [1]:
%pip install ipyfilechooser ipywidgets


Note: you may need to restart the kernel to use updated packages.


## Code Hiding

In [2]:
import ipywidgets as widgets
from IPython.display import display, HTML
# ========== TOGGLE BTN ==========
javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """
    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)
    display(HTML(output))

def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new
    toggle_code(state)
    value.owner.description = button_descriptions[state]
    
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")
# ========== END TOGGLE BTN END ==========
display(button)

ToggleButton(value=False, description='Show code')

# Data Exploration

### T10-4
As a USER, I want to have a Data Exploration Section in the notebook so that I can easily locate the upload and playback functions in the notebook.

# Inference

### T10-7
As a USER, I want to have an Inference Section in the notebook so that I can easily locate the Inference Section in the notebook.

Output video processing functions

In [3]:
import cv2

def output(video):
    cap = cv2.VideoCapture(video)
    # Check if camera opened
    if not cap.isOpened():
        print("Error opening video file")

    # Get video height and width
    # frame_width = int(cap.get(3))
    # frame_height = int(cap.get(4))

    # Define output file name and format, codec used, define fps and the frame size
    # output_video = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 30.0,
    #                                (frame_width,frame_height))

    try:
        while cap.isOpened():
            # Capture frame by frame
            ret, frame = cap.read()

            # Check if there are frames captured
            if not ret:
                # Destroy frame window
                cv2.destroyAllWindows()
                # Release the Video Device
                cap.release()
                break

            # writing the new frame in output
            # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # pyplot.axis('off')
            # pyplot.imshow(frame)
            # pyplot.title('output')
            # pyplot.show()
            # clear_output(wait=True)

            # Write camera capture into video
            # output_video.write(frame)

            # Display video in a frame
            cv2.imshow('Frame', frame)

            # Press Q to quit player
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
                
        # Release the Video Device
        cv2.destroyAllWindows()
        cap.release()
        # Message to be displayed after releasing the device
        print("Released Video Resource")

    # To catch when stopping cell run in notebook
    except KeyboardInterrupt:
        cv2.destroyAllWindows()
        cap.release()
        # Message to be displayed after releasing the device
        print("Released Video Resource")

if __name__ == "__main__":
    output("") # video path and file here

Error opening video file
Released Video Resource


### T10-6
As a USER, I want to have a Training Section in the notebook so that I can easily locate the training related functions in the notebook.

## File Upload

In [4]:
import ipywidgets as widgets
from IPython.display import display, HTML
from ipyfilechooser import FileChooser
import zipfile as zf

# ========== FileChooser ==========
# Create and display a FileChooser widget
fc = FileChooser('/Users')

# Change defaults and reset the dialog
# fc.default_path = './'
# fc.reset()

# Restrict navigation to /Users
fc.sandbox_path = '/Users'

# Change hidden files
fc.show_hidden = False

# Switch to folder-only mode
fc.show_only_dirs = False

# Set multiple file filter patterns (uses https://docs.python.org/3/library/fnmatch.html)
# fc.filter_pattern = ['*.jpg', '*.png','*.txt']
fc.filter_pattern = ['*.zip']

# Change the title (use '' to hide)
fc.title = '<b>Select File (*zip)</b>'

# Sample callback function
def change_display_selected(chooser):
    print(fc.selected_filename, end='\r')

# Register callback function
fc.register_callback(change_display_selected)

def on_button_clicked(b):
    # FILE UPLOAD
    print(fc.selected)
    fn = fc.selected_filename.replace('.zip', '')
    files = zf.ZipFile(fc.selected_path+"\\"+fc.selected_filename, 'r')
    files.extractall('Data Folder\\'+fn)
    files.close()
    with output:
        print("Files saved.")
# ========== END FileChooser END ==========
# ========== CONFIRM BTN ==========
confirmBtn = widgets.Button(
    description='Confirm Upload',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Confirm',
)
output = widgets.Output()
confirmBtn.on_click(on_button_clicked)
# ========== CONFIRM BTN ==========
display(fc)
display(confirmBtn, output)

InvalidPathError: /Users does not exist

## Select File for Training

In [1]:
from IPython import display
from ipyfilechooser import FileChooser
from IPython.display import display
from IPython.display import IFrame
import ipywidgets as widgets

output2 = widgets.Output()
# ========== FileChooser ==========
# Create and display a FileChooser widget
fc2 = FileChooser('./')
display(fc2)

# Restrict navigation to /Users
fc2.sandbox_path = './'

# Change hidden files
fc2.show_hidden = False

# Switch to folder-only mode
fc2.show_only_dirs = False

# Change the title (use '' to hide)
fc2.title = '<b>Select File for Training</b>'

# Sample callback function
def change_display_selected(chooser):
    with output2:
        print(fc2.selected_filename, end='\r')
    
# Register callback function
fc2.register_callback(change_display_selected)
# ========== END FileChooser END ==========
display(output2)

FileChooser(path='D:\3104 - TSU\ict3104-team10-2022', filename='', title='', show_hidden=False, select_desc='S…

Output()

In [2]:
!pip freeze

absl-py==1.1.0
alembic==1.6.5
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.0.5
astunparse==1.6.3
attrs==21.4.0
autopep8==1.6.0
backcall==0.2.0
backports.entry-points-selectable==1.1.0
beautifulsoup4==4.11.1
bleach==5.0.1
cachetools==5.2.0
certifi==2021.10.8
cffi==1.15.0
charset-normalizer==2.0.12
click==8.0.3
colorama==0.4.4
cycler==0.11.0
debugpy==1.6.0
decorator==5.1.1
defusedxml==0.7.1
distlib==0.3.6
dominate==2.7.0
entrypoints==0.4
executing==0.8.3
fastjsonschema==2.15.3
filelock==3.8.0
Flask==2.0.2
Flask-Bootstrap==3.3.7.1
Flask-Cors==3.0.10
flask-marshmallow==0.14.0
Flask-Migrate==3.1.0
Flask-SQLAlchemy==2.5.1
flatbuffers==1.12
fonttools==4.32.0
gast==0.4.0
google-auth==2.9.0
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
greenlet==1.1.2
grpcio==1.47.0
gunicorn==20.1.0
h5py==3.7.0
idna==3.3
ipyfilechooser==0.6.0
ipykernel==6.13.0
ipython==8.4.0
ipython-genutils==0.2.0
ipywidgets==7.7.1
itsdangerous==2.0.1
jedi==0.18.1
Jinja2==3.0.2
joblib==1.1.0
jsonschema==4.6.